In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [5]:
# Python < 3.8?
import sys
#! pip install pickle5

if sys.version_info.minor < 8:
    import pickle5 as pickle

In [6]:
import sys
sys.path.append("../")
from registration_tree import Rect, QuadTree

In [25]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '#####' #'exact'
configuration.host = "https://exact.cs.fau.de/" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [17]:
Path(qtree.source_slide_name).name

'CRC-AI-1 D X40 IHC.tif'

In [27]:
images_api.list_images(name=Path(qtree.source_slide_name).name, image_set=304)

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'annotations': 'Annotations not load please remove '
                             'omit=annotations',
              'filename': 'CRC-AI-1 D X40 IHC.tif',
              'height': 74256,
              'id': 10440,
              'image_set': 304,
              'image_type': 0,
              'mpp': 0.25,
              'name': 'CRC-AI-1 D X40 IHC.tif',
              'objective_power': 40.0,
              'time': datetime.datetime(2021, 1, 15, 16, 20, 33, 518979),
              'width': 86304}]}

In [15]:
list(Path("Depth_2/2048").glob("*-To-*.pickle"))

[WindowsPath('Depth_2/2048/CRC-AI-1 D X40 IHC-To-CRC-A1-1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-10 X40 IHC-To-CRC-A1-10 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-11 1D X40 IHC-To-CRC-AI-11 D1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-11 F X40 IHC-To-CRC-AI-11 F1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-11 H X40 IHC-To-CRC-AI-11 H1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-13 X40 IHC-To-CRC-A1-13 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-14 1E X40 IHC-To-CRC-AI-14 E1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-14 1F X40 IHC-To-CRC-AI-14 F1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-14 1G X40 IHC-To-CRC-AI-14 G1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-14 1H X40 IHC-To-CRC-AI-14 H1 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-15 X40 IHC-To-CRC-AI-15 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-2 X40 IHC-To-CRC-A1-2 40X.pickle'),
 WindowsPath('Depth_2/2048/CRC-AI-3 D X40 IHC-To-CRC-AI-3 D1 40X.pickle'),
 WindowsPath('Depth_2/2

In [39]:
qtrees = {}

for path in Path("Depth_2/2048").glob("*-To-*.pickle"):
    
    source, target = path.stem.split("-To-")
    qtree = pickle.load(open(str(path), "rb"))

    if source not in qtrees:
        qtrees[source] = {}
    
    source_image = images_api.list_images(name=Path(qtree.source_slide_name).name, image_set=304).results[0]
    target_image = images_api.list_images(name=Path(qtree.target_slide_name).name, image_set=304).results[0]
    
    qtrees[source] = { 
        "qtree": qtree,
        "source_image": source_image,
        "target_image": target_image
    }

2021-01-28 11:08:06,587 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))': //api/v1/images/images/?name=CRC-AI-1+D+X40+IHC.tif&image_set=304&omit=annotations


In [40]:
qtrees

{'CRC-AI-1 D X40 IHC': {'qtree': <registration_tree.QuadTree at 0x248ce32e588>,
  'source_image': {'annotations': 'Annotations not load please remove omit=annotations',
   'filename': 'CRC-AI-1 D X40 IHC.tif',
   'height': 74256,
   'id': 10440,
   'image_set': 304,
   'image_type': 0,
   'mpp': 0.25,
   'name': 'CRC-AI-1 D X40 IHC.tif',
   'objective_power': 40.0,
   'time': datetime.datetime(2021, 1, 15, 16, 20, 33, 518979),
   'width': 86304},
  'target_image': {'annotations': 'Annotations not load please remove omit=annotations',
   'filename': 'CRC-A1-1 40X.tif',
   'height': 78128,
   'id': 10419,
   'image_set': 304,
   'image_type': 0,
   'mpp': 0.25,
   'name': 'CRC-A1-1 40X.tif',
   'objective_power': 40.0,
   'time': datetime.datetime(2021, 1, 14, 9, 13, 50, 234532),
   'width': 81760}},
 'CRC-AI-10 X40 IHC': {'qtree': <registration_tree.QuadTree at 0x248cec82278>,
  'source_image': {'annotations': 'Annotations not load please remove omit=annotations',
   'filename': 'CRC-AI

In [33]:
source_image

{'annotations': 'Annotations not load please remove omit=annotations',
 'filename': 'CRC-AI-1 D X40 IHC.tif',
 'height': 74256,
 'id': 10440,
 'image_set': 304,
 'image_type': 0,
 'mpp': 0.25,
 'name': 'CRC-AI-1 D X40 IHC.tif',
 'objective_power': 40.0,
 'time': datetime.datetime(2021, 1, 15, 16, 20, 33, 518979),
 'width': 86304}

In [37]:
item

{'CRC-A1-1 40X': {'qtree': <registration_tree.QuadTree at 0x248ce32ec88>,
  'source_image': {'annotations': 'Annotations not load please remove omit=annotations',
   'filename': 'CRC-AI-1 D X40 IHC.tif',
   'height': 74256,
   'id': 10440,
   'image_set': 304,
   'image_type': 0,
   'mpp': 0.25,
   'name': 'CRC-AI-1 D X40 IHC.tif',
   'objective_power': 40.0,
   'time': datetime.datetime(2021, 1, 15, 16, 20, 33, 518979),
   'width': 86304},
  'target_image': {'annotations': 'Annotations not load please remove omit=annotations',
   'filename': 'CRC-A1-1 40X.tif',
   'height': 78128,
   'id': 10419,
   'image_set': 304,
   'image_type': 0,
   'mpp': 0.25,
   'name': 'CRC-A1-1 40X.tif',
   'objective_power': 40.0,
   'time': datetime.datetime(2021, 1, 14, 9, 13, 50, 234532),
   'width': 81760}}}

In [38]:
name

'CRC-AI-1 D X40 IHC'

In [53]:
urls = []
annotations_to_upload = []

for name, item in qtrees.items():
    
    qtree = item["qtree"]
    source_image = item["source_image"]
    target_image = item["target_image"]
    
    for anno in annotations_api.list_annotations(image=source_image.id, deleted=False, pagination=False,
                                                 fields="annotation_type,id,image,vector,unique_identifier",
                                                 ).results:
        
        xmin, ymin, xmax, ymax = anno.vector["x1"], anno.vector["y1"], anno.vector["x2"], anno.vector["y2"]
        source_url = f"{configuration.host}annotations/{source_image.id}/?xmin={xmin}&ymin={ymin}&xmax={xmax}&ymax={ymax}"

        
        w, h = xmax - xmin, ymax - ymin
        x_c, y_c = xmin + w // 2, ymin + h // 2
        
        box = [x_c, y_c, w, h]
        trans_box = qtree.transform_boxes(np.array([box]))[0]
        
        new_x1 = int(trans_box[0] - trans_box[2] // 2)
        new_y1 = int(trans_box[1] - trans_box[3] // 2)
        new_x2 = int(trans_box[0] + trans_box[2] // 2)
        new_y2 = int(trans_box[1] + trans_box[3] // 2)

        vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}
        target_url = f"{configuration.host}annotations/{target_image.id}/?xmin={new_x1}&ymin={new_y1}&xmax={new_x2}&ymax={new_y2}"
        
        urls.append((source_url, target_url))
        
        
        anno = Annotation(annotation_type=anno.annotation_type, 
                          vector=vector, 
                          unique_identifier=anno.unique_identifier,
                          image=target_image.id)

        annotations_to_upload.append(anno)    

2021-01-28 11:20:19,523 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))': //api/v1/annotations/annotations/?deleted=False&image=10440&fields=annotation_type%2Cid%2Cimage%2Cvector%2Cunique_identifier


https://exact.cs.fau.de/annotations/10426/?xmin=15044&ymin=83533&xmax=15472&ymax=83789
https://exact.cs.fau.de/annotations/10448/?xmin=14770&ymin=81341&xmax=15194&ymax=81594
    
    
https://exact.cs.fau.de/annotations/10434/?xmin=5653&ymin=25371&xmax=6015&ymax=25587
https://exact.cs.fau.de/annotations/10457/?xmin=6385&ymin=25777&xmax=6747&ymax=25993

In [59]:
import random

for source_url, target_url in random.choices(urls, k=5):
    print(f"{source_url} ->  {target_url}") 

https://exact.cs.fau.de/annotations/10453/?xmin=3528&ymin=51917&xmax=3839&ymax=52161 ->  https://exact.cs.fau.de/annotations/10431/?xmin=2304&ymin=52897&xmax=2614&ymax=53141
https://exact.cs.fau.de/annotations/10445/?xmin=10150&ymin=35466&xmax=10470&ymax=35722 ->  https://exact.cs.fau.de/annotations/10418/?xmin=13617&ymin=37713&xmax=13937&ymax=37967
https://exact.cs.fau.de/annotations/10439/?xmin=25247&ymin=14390&xmax=25443&ymax=14650 ->  https://exact.cs.fau.de/annotations/10421/?xmin=30675&ymin=57167&xmax=30869&ymax=57427
https://exact.cs.fau.de/annotations/10457/?xmin=14104&ymin=13740&xmax=14205&ymax=13913 ->  https://exact.cs.fau.de/annotations/10434/?xmin=13341&ymin=13319&xmax=13441&ymax=13491
https://exact.cs.fau.de/annotations/10457/?xmin=6385&ymin=25785&xmax=6747&ymax=25985 ->  https://exact.cs.fau.de/annotations/10434/?xmin=5653&ymin=25380&xmax=6015&ymax=25578


In [60]:
annotations_to_upload[:5]

[{'annotation_type': 364,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10419,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '57515035-c163-4fd4-b058-f18dadc3b1dc',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 3162, 'x2': 3342, 'y1': 68085, 'y2': 68275},
  'verified_by_user': None},
 {'annotation_type': 364,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10419,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'ca9f7d32-9243-4429-a7eb-d01b1b475420',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 18152, 'x2': 18282, 'y1': 32699, 'y2': 33003},
  'verified_by_user': None},
 {'annotation_type': 364,
  'annotationversion_set': [],
  'blurred': None,
  'c

In [62]:
annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 364,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2769649,
  'image': 10419,
  'last_edit_time': datetime.datetime(2021, 1, 28, 11, 42, 16, 137199),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 28, 11, 42, 16, 137126),
  'unique_identifier': '57515035-c163-4fd4-b058-f18dadc3b1dc',
  'uploaded_media_files': [],
  'user': 1,
  'vector': {'x1': 3162, 'x2': 3342, 'y1': 68085, 'y2': 68275},
  'verified_by_user': 'False'},
 {'annotation_type': 364,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2769650,
  'image': 10419,
  'last_edit_time': datetime.datetime(2021, 1, 28, 11, 42, 16, 137431),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 28, 11, 42, 16, 137384),
  'unique_identifier': 'ca9f7d32-9243-4429-a7eb-d01b1b475420',
  'uploaded_media_files': [